In [16]:
import chromadb
client = chromadb.PersistentClient('data/chroma-data')
#client = chromadb.HttpClient(host='localhost', port=8000)

In [17]:
texts_collection = client.create_collection(name='eidc_texts')
sentences_collection = client.create_collection(name='eidc_sentences')

In [18]:
from util import load_eidc_data
df = load_eidc_data.load_as_df('data/catalogue_metadata.json')
df = df.drop_duplicates(subset=['title'])

In [23]:
import nltk
eidc_sentences = []
eidc_sentence_ids = []
for i, row in df.iterrows():
    sent_count = 0
    for sentence in nltk.sent_tokenize(row['description']):
        eidc_sentences.append(sentence)
        eidc_sentence_ids.append(f"{row['title']}_{sent_count}")
        sent_count += 1
print(f'{len(eidc_sentences)}:{len(eidc_sentence_ids)}')

11607:11607


In [20]:
eidc_docs = df['description'].to_list()
eidc_ids = df['title'].to_list()
print(f'{len(eidc_docs)}:{len(eidc_ids)}')


1864:1864


In [21]:
texts_collection.add(
    documents=eidc_docs,
    ids=eidc_ids
)

In [24]:
sentences_collection.add(
    documents=eidc_sentences,
    ids=eidc_sentence_ids
)

In [25]:
queries = ['Where is the wettest soil in the UK?',
             'Where is water quality worst in the UK?',
             'Where are bird populations declining in the UK?',
             'Where in the UK are bumblebees most at risk from neonicotinoids?',
             'Which county in the UK has the most rivers?']

In [26]:
results = texts_collection.query(
    query_texts = queries,
    n_results = 5,
    include = ['distances']
)
results

{'ids': [['Topsoil physico-chemical properties from the UKCEH Countryside Survey, Great Britain, 2020, v2',
   'Topsoil physico-chemical properties from the UKCEH Countryside Survey, Great Britain, 2021',
   'Topsoil physico-chemical properties from the UKCEH Countryside Survey, Great Britain, 2018-2019',
   'Topsoil physico-chemical properties from the UKCEH Countryside Survey, Great Britain, 2022',
   'Topsoil physico-chemical properties from the UKCEH Countryside Survey, Great Britain, 2019'],
  ['Water quality data from the Ribble and Wyre catchments 2008-2010',
   'Weekly water quality data from the River Thames and its major tributaries (2009-2017)',
   'Weekly water quality data from the River Thames and its major tributaries (2009-2013) [CEH Thames Initiative]',
   'Nutrient chemistry data from five rivers receiving sewage effluent in Oxfordshire, England, 2017',
   'Thames catchment hydrometric and water quality data 2013-17'],
  ['Diet, timing of egg laying and breeding succe

In [27]:
results = sentences_collection.query(
    query_texts = queries,
    n_results = 5,
    include = ['distances']
)
results

{'ids': [['Soil physico-chemical properties 2007 [Countryside Survey]\t_0',
   'Daily and sub-daily hydrometeorological and soil data (2013-2023) [COSMOS-UK]_1',
   'Daily and sub-daily hydrometeorological and soil data (2013-2022) [COSMOS-UK]_1',
   'Daily and sub-daily hydrometeorological and soil data (2013-2019) [COSMOS-UK]_1',
   'Daily and sub-daily hydrometeorological and soil data (2013-2018) [COSMOS-UK]_1'],
  ['Water quality data from the Ribble and Wyre catchments 2008-2010_0',
   'Thames catchment hydrometric and water quality data 2013-17_0',
   'Rainfall, throughfall and stemflow for beech and ash stands in temperate English deciduous woodlands_1',
   'Faecal indicator data from 54 lakes in Greater Glasgow, Cumbria, and Norfolk, UK, 2016-2017_0',
   'Water quality data for the River Frome catchment, Dorset, 1976-2022_0'],
  ['UK Environmental Change Network (ECN) common breeding birds data 1971-2007_0',
   'UK Environmental Change Network (ECN) bird data: 1995-2015_0',
  

In [30]:
from tabulate import tabulate
md = tabulate(results['ids'])
print(md)

-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------  --------------------------------------------------------------------------------------------------------------------  -----------------------------------------------------------------------------------------------------------------------  ---------------------------------------------------------------------------------------------------------------------------------  --------------------------------------------------------------------------------------------------------------------------------
Soil physico-chemical properties 2007 [Countryside Survey]	_0                                                                                                                                                                                      Daily and sub-daily hydrometeorological and soil data (2013-2023) [COSMOS-UK]